In [14]:
import pandas as pd
from sqlite3 import connect
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn import metrics
from sklearn import decomposition
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import batch_normalization
import tensorflow as tf



In [3]:
conn = connect("data/dataset_review.db")
dataset = pd.read_sql("SELECT * FROM games", conn)
conn.close()

dataset.set_index("index", inplace=True)
dataset = dataset.loc[dataset["summary"].notnull()]
dataset = dataset.loc[dataset["reviews"].notnull()]
dataset = dataset[dataset["user_review"] != -1]
dataset.reset_index(drop=True, inplace=True)

In [4]:
# tfv = TfidfVectorizer(min_df=3,  max_features=None, 
#             strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
#             ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
#             stop_words = 'english')
# tfv.fit(dataset["summary"].str.lower())
# summary = tfv.transform(dataset["summary"])
# kmeans = KMeans(n_clusters=10, random_state=7).fit(summary)
# summary_kmeans = np.array(kmeans.labels_)
# np.save("data/summary_kmeans.npy", summary_kmeans)

summary_kmeans = np.load("data/summary_kmeans.npy", allow_pickle=True)

In [5]:
categorical_labels = to_categorical(summary_kmeans, num_classes=10)
dataset = pd.concat([dataset, pd.DataFrame(categorical_labels, columns=["summary_kmean_" + str(i) for i in range(10)])], axis=1)
y = dataset["user_review"]
x = dataset.drop(columns=["user_review"])

In [6]:

tfv = CountVectorizer(analyzer='word', stop_words = 'english')
a = tfv.fit_transform(dataset["reviews"].values)

In [7]:
sc = preprocessing.StandardScaler(with_mean=False)
svd = decomposition.TruncatedSVD(n_components=700)
b = svd.fit_transform(sc.fit_transform(a))

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(b, y, random_state=7, test_size=0.2, shuffle=True)

In [ ]:
regr = make_pipeline(preprocessing.StandardScaler(with_mean=False), decomposition.TruncatedSVD(n_components=700), SVR(C=1.0, epsilon=0.2))
regr.fit(x_train, y_train)
pred = regr.predict(x_valid)
print(metrics.r2_score(y_true=y_valid, y_pred=pred))





In [ ]:
x_train.shape

In [57]:
model = Sequential()

model.add(Dense(300, input_dim=700, activation='relu'))
model.add(Dropout(0.2))
model.add(batch_normalization.BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(batch_normalization.BatchNormalization())

model.add(Dense(1))
model.add(Activation('relu'))

model.compile(loss='mse', optimizer='adam')

In [58]:
model.fit(x_train, y=y_train.ravel(), batch_size=64, 
          epochs=100, verbose=1, 
          validation_data=(x_valid, y_valid.ravel()))

Epoch 1/100
186/186 [==============================] - 1s 4ms/step - loss: 0.2887 - val_loss: 0.1564
Epoch 2/100
186/186 [==============================] - 1s 4ms/step - loss: 0.1561 - val_loss: 0.0904
Epoch 3/100
186/186 [==============================] - 1s 3ms/step - loss: 0.1203 - val_loss: 0.0709
Epoch 4/100
186/186 [==============================] - 1s 4ms/step - loss: 0.0964 - val_loss: 0.0595
Epoch 5/100
186/186 [==============================] - 1s 3ms/step - loss: 0.0750 - val_loss: 0.0475
Epoch 6/100
186/186 [==============================] - 1s 3ms/step - loss: 0.0614 - val_loss: 0.0363
Epoch 7/100
186/186 [==============================] - 1s 3ms/step - loss: 0.0460 - val_loss: 0.0229
Epoch 8/100
186/186 [==============================] - 1s 4ms/step - loss: 0.0308 - val_loss: 0.0172
Epoch 9/100
186/186 [==============================] - 1s 4ms/step - loss: 0.0246 - val_loss: 0.0212
Epoch 10/100
186/186 [==============================] - 1s 3ms/step - loss: 0.0203 - val_lo

In [59]:
pred_nn = model.predict(x_valid)

In [60]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.truediv(residual, total))
  return r2

In [61]:
print(R_squared(y_valid.values, pred_nn.ravel()))

tf.Tensor(0.22562951471312076, shape=(), dtype=float64)
